#  NWB File generation from raw data

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import logging
import sys
from rec_to_nwb.processing.builder.nwb_file_builder import NWBFileBuilder
from rec_to_binaries import extract_trodes_rec_file
from rec_to_nwb.processing.builder.raw_to_nwb_builder import RawToNWBBuilder
from rec_to_nwb.processing.metadata.metadata_manager import MetadataManager

## Setup config parameters
### see more details at rec_to_nwb repo readme or [here](https://novelaneuro.github.io/rec_to_nwb-docs/README.html#how-to-use-it)

In [ ]:
# set the animal name and the date or list of dates to process
animal_name = 'beans'
dates = ['20190718']
# note that code currently only takes one date at a time;
# build a loop through date strings to create multiiple nwb files at once

# path to dir containing device and animal/date .yml files
# for example, '/home/<yourusername>/Src/franklabnwb/yaml'
yaml_path = '/home/loren/Src/NWB/franklabnwb/yaml'
# metadata filename for an animal and date. This is a file that you generate as the user.
# for example,  '<ratname><date>_metadata.yml'
animal_metadata_file = 'beans20190718_metadata.yml' #bad_channels in yaml will be left out of the .nwb filie
# metadata filename for a recording device type;
# typically these are common lab files that you shouldn't need to personalize
probe1_metadata_file = '128c-4s8mm6cm-20um-40um-sl.yml'
probe2_metadata_file = 'tetrode_12.5.yml'

# Specify the paths for the data, the output nwb file, and the video files
# example raw data INPUT path: '/stelmo/<yourusername>/'
data_path = '/stelmo/loren/'
#  nwb file OUTPUT path: '/stelmo/nwb/raw/'
# all NWB files should go here
output_path='/stelmo/nwb/raw/'
#  video OUTPUT path: '/stelmo/nwb/video/'
# just copies over the raw video files into this dir
video_path='/stelmo/nwb/video/'
# note that a rec_to_nwb.log file will also be saved to the directory from which your are running this notebook

# Specify any optional trodes export flags
# uses rec_to_binaries repo, which uses SpikeGadgets/Trodes export functions
# keep things as raw as possible though - unlike old preprocessing pipeline, 
# we aren't doing any filtering or interpolation yet
trodes_rec_export_args = ('-reconfig', '/stelmo/loren/beans/Probe_128ch_allnT_DIOs_PTP_reconfig_export_shanks.xml') 

# specify the locations of the metadata files for the animal and the probe(s). 
# Note that specifying all possible probes is fine
animal_metadata = os.path.join(yaml_path, animal_metadata_file )
probe_metadata = [os.path.join(yaml_path, probe1_metadata_file), 
                  os.path.join(yaml_path, probe2_metadata_file)]

# Specify whether preprocessing data should be reextracted (True) or skipped if it already exists (False)
overwrite=False
# note that extraction will write to the directory: data_path+'preprocessing/' and
# you may have data you previously preprocessed in that dir, created with alternate export args
# that you neither want to skip nor overwrite if you're using them for filterframework


In [ ]:
### Print the meta data as a check

In [ ]:
metadata = MetadataManager(animal_metadata, probe_metadata)
print(metadata)


# Processing initiation

In [ ]:
builder = RawToNWBBuilder(animal_name=animal_name,
                          data_path=data_path,
                          dates=dates,
                          nwb_metadata=metadata,
                          overwrite=overwrite,
                          output_path=output_path,
                          video_path=video_path,
                          trodes_rec_export_args = trodes_rec_export_args)

## Run processing and create nwb file

In [ ]:
%%time
content = builder.build_nwb()
print(content)
#expect warnings from rec_to_binaries


# Cleanup preprocessed data
## Note that this deletes the extracted data, so you may wish to skip this step

In [ ]:
builder.clean_up()

## Test: Import the NWB file and inspect the contents

In [ ]:
import pynwb
nwb_file_name = output_path + animal_name + dates[0] + '.nwb'
# such as: '/stelmo/<yourusername>/<ratname>/nwbout/<ratname><experimentdate>.nwb'
io = pynwb.NWBHDF5IO(nwb_file_name,'r')
nwbf = io.read()


In [ ]:
nwbf

In [ ]:
lst = [12] 
",".join([str(i) for i in lst])

In [ ]:
640 * 2 * 30000 * 60 * 60 * 24 *30 / 1e12